In [1]:
import os
os.chdir("/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali")
os.environ["HF_HOME"] = "/media/pc1/Ubuntu/Extend_Data/hf_models"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import pandas as pd
from src.rag.vectordb.middleware import Middleware

output_folder = f"data/processed/1/"
len(os.listdir(output_folder))

/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


./config/rag_config.yaml
./config/rag_config.yaml
./config/rag_config.yaml


6471

In [3]:
middleware = Middleware(
    kg_path="data/processed/knowledge graph of DDXPlus.xlsx",
    model_name="vidore/colqwen2-v1.0", 
    quantized=True, 
    create_collection=True,
    enable_rag=True,
    quantize_llm=False,
    quantization_type_llm="4bit"
)

Loading checkpoint shards: 100%|██████████| 2/2 [01:00<00:00, 30.48s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:46<00:00, 23.43s/it]
Device set to use cuda:0


# Indexing

In [11]:
path = "data/raw/clinical_cases"
files = os.listdir(path)
for file in files:
    file = os.path.join(path, file)
    middleware.index("1", file)

[Middleware] Indexing data/raw/clinical_cases/e545-Cervical-and-lumbar-diffuse-Brucella-spondylodiscitis.pdf
The number of unprocessed images is: 0
  • Saved 4 page images
Processing 4 images


  0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt: 

# RAG

In [4]:
import json

path_file = "data/raw/cases_test"
files = os.listdir(path_file)
text_lst = []
img_lst = []

for file in files:
    file_path = os.path.join(path_file, file)
    imgs = []

    # Load JSON
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Combine all fields except "Extracted Figure"
    combined_text = ""
    for key, value in data.items():
        if key not in ["Extracted Figure", "Further Reading"]:
            combined_text += f"{key}:\n{value}\n\n"

    # Add Descriptions from "Extracted Figure"
    descriptions = []
    for figure_group in data.get("Extracted Figure", []):
        for figure in figure_group:
            desc = figure.get("Description", "")
            descriptions.append(desc)

    if descriptions:
        combined_text += "Figure Descriptions:\n"
        for desc in descriptions:
            combined_text += f"{desc}\n"

    # Get image links
    image_paths = []
    for figure_group in data.get("Extracted Figure", []):
        for figure in figure_group:
            path = figure.get("Link")
            path = os.path.join('data/raw', path)
            if path:
                image_paths.append(path)

    # Final combined text
    text_lst.append(combined_text)
    img_lst.append(image_paths)


# Testing

In [5]:
import json
import os

def safe_append_json(filepath, record):
    # Ensure the file exists with a valid empty JSON array
    if not os.path.exists(filepath):
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump([], f)

    # Load safely, reset if corrupted
    with open(filepath, "r+", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if not isinstance(data, list):
                data = []  # Force reset if JSON is not a list
        except json.JSONDecodeError:
            data = []  # Reset if file is empty or invalid

        data.append(record)

        # Overwrite with updated JSON
        f.seek(0)
        json.dump(data, f, indent=4, ensure_ascii=False)
        f.truncate()

In [6]:
# output_file = "data/processed/results.json"
# processed_files = []
# with open(output_file, "r", encoding="utf-8") as f:
#     data = json.load(f)
# for case in data:
#     for key, value in case.items():
#             if key in ['original_file']:
#                 processed_files.append(value)

In [7]:
for query, imgs, file in zip(text_lst, img_lst, files):
    # if file not in processed_files:
    vectordb_docs = middleware._search_vectordb(query=query, top_k=5)
    results_kg, histories, symptoms = middleware._search_knowledge_graph(query=query, top_k=1)
    pdf_paths = [pdf_path['original_file'] for pdf_path in vectordb_docs] 
    answer = middleware.get_answer_from_medgemma(
        query=query,
        images_path=imgs if imgs else [],
        retrieved_docs=pdf_paths,
        kg_info=results_kg,
    )
    print(f"File: \n{file}")
    print(f"Vector DB: \n{vectordb_docs}")
    print(f"KG results: \n{results_kg}")

    # record = {
    #     "query": query,
    #     "images_used": imgs,
    #     "pdf_paths": pdf_paths,
    #     "results_kg": results_kg,
    #     "answer": answer,
    #     "original_file": file
    # }
    break

        # # Load existing data, append, and rewrite
        # safe_append_json(output_file, record)


--------------------------------------------------------------------------------
Searching in vector database
Processing 8649 texts
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Searching in Knowledge Graph


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Feature Decompostion: 
{'history': ['abdominal pain', 'chest pain', 'cough', 'diarrhoea', 'fever', 'nausea', 'severe asthenia', 'vomiting'], 'symptoms': ['abdominal tenderness', 'bleeding', 'coma', 'copious bleeding', 'dyspnoea', 'hepatosplenomegaly', 'hypothermic shock', 'prostrate', 'semiconscious', 'shock', 'subconjunctival haemorrhage']}
Relevant Level 3 Descriptions: ['myocarditis', 'pericarditis']
--------------------------------------------------------------------------------
File: 
1---A-20-Year-Old-Woman-from-Sudan-With-Fever--_2022_Clinical-Cases-in-Tropi.json
Vector DB: 
[{'original_file': 'data/raw/clinical_cases/NEJMcpc2107356.pdf'}, {'original_file': 'data/raw/clinical_cases/NEJMcpc2211421.pdf'}, {'original_file': 'data/raw/clinical_cases/NEJMcpc2211363.pdf'}, {'original_file': 'data/raw/clinical_cases/NEJM199504133321508.pdf'}, {'original_file': 'data/raw/clinical_cases/NEJMcpc2309498.pdf'}]
KG results: 
myocarditis has symptomatology Chest pain, fatigue, shortness of br

In [ ]:
results = middleware.search(example_query, 5)

[Middleware] Received query/queries
Query: '\n42‑year‑old Vietnamese man presents with a 4‑day history of high‑grade fever\n(peak 39.5\u202f°C), productive cough with yellow‑green sputum, and night sweats.\nHe notes a 5\u202fkg unintended weight‑loss over the past two months.\n\n• Travel / exposure: returned from Tây\u202fNinh Province 2\u202fweeks ago after working\n  on a pig farm; lives in Ho\u202fChi\u202fMinh City.\n• Past history: treated pulmonary tuberculosis 5\u202fyears ago; 10\u202fpack‑year smoker;\n  no known HIV; no chronic illnesses.\n• Vitals: BP\u202f110/70\u202fmmHg, HR\u202f108\u202fbpm, RR\u202f24/min, Temp\u202f38.8\u202f°C, SpO₂\u202f93\u202f%\u202fRA.\n• Exam: crackles over the right upper‑lobe; no wheeze.\n• Labs: WBC\u202f14\u202f×\u202f10⁹/L (neutrophils\u202f80\u202f%), Hb\u202f12\u202fg/dL, random glucose\u202f7.2\u202fmmol/L.\n• Imaging: chest X‑ray attached (suspected RUL cavitation).\n'
---------------------------------------------------------------------

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Relevant Level 3 Descriptions: ['atrial_fibrillation', 'myocarditis', 'pericarditis', 'psvt', 'possible_nstemi_stemi', 'stable_angina', 'unstable_angina']
Relevant Level 3 Descriptions: ['pulmonary_embolism', 'acute_copd_exacerbation_infection', 'bronchiectasis', 'bronchiolitis', 'bronchitis', 'bronchospasm_acute_asthma_exacerbation', 'pulmonary_neoplasm', 'urti', 'viral_pharyngitis', 'whooping_cough', 'acute_laryngitis', 'acute_pulmonary_edema', 'croup', 'larygospasm', 'epiglottitis']
Relevant Level 3 Descriptions: ['pneumonia', 'tuberculosis', 'hiv_initial_infection', 'ebola', 'influenza', 'chagas', 'acute_otitis_media', 'acute_rhinosinusitis', 'chronic_rhinosinusitis']
Relevant Level 3 Descriptions: ['gerd', 'boerhaave_syndrome', 'pancreatic_neoplasm', 'scombroid_food_poisoning']
Relevant Level 3 Descriptions: ['myasthenia_gravis', 'guillain_barre_syndrome', 'cluster_headache', 'acute_dystonic_reactions']
Additional Info: atrial_fibrillation has symptomatology Irregular heartbeat, p

In [ ]:
results_vectordb, results_kg = results

In [ ]:
results_vectordb

[{'filepath': 'data/processed/1//page_1.png',
  'original_file': 'data/raw/clinical_cases/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T.pdf',
  'score': 52.0},
 {'filepath': 'data/processed/1//page_1.png',
  'original_file': 'data/raw/clinical_cases/26---A-14-Year-Old-Boy-from-Malawi-Who-Has-Bee_2022_Clinical-Cases-in-Tropic.pdf',
  'score': 50.0},
 {'filepath': 'data/processed/1//page_1.png',
  'original_file': 'data/raw/clinical_cases/37---A-29-Year-Old-Woman-from-Malawi-With-Confusi_2022_Clinical-Cases-in-Tro.pdf',
  'score': 50.0},
 {'filepath': 'data/processed/1//page_1.png',
  'original_file': 'data/raw/clinical_cases/11---A-45-Year-Old-Male-Security-Guard-from-Malawi-_2022_Clinical-Cases-in-T.pdf',
  'score': 50.0},
 {'filepath': 'data/processed/1//page_1.png',
  'original_file': 'data/raw/clinical_cases/48---A-31-Year-Old-Woman-from-Tanzania-With-Acu_2022_Clinical-Cases-in-Tropi.pdf',
  'score': 48.0}]

In [ ]:
image_paths = [retrieved_img['filepath'] for retrieved_img in results_vectordb] 

In [ ]:
image_paths

['data/processed/1//page_1.png',
 'data/processed/1//page_1.png',
 'data/processed/1//page_1.png',
 'data/processed/1//page_1.png',
 'data/processed/1//page_1.png']

In [ ]:
answer = middleware.get_answer_from_medgemma(example_query, image_paths, results_kg)

In [ ]:
print(answer)

Based on the provided information, the most likely diagnosis is **Ebola Virus Disease**.

Here's why:

*   **Travel History:** The patient's recent travel to Tây Ninh Province, Vietnam, is a significant risk factor for Ebola.
*   **Symptoms:** The patient presents with fever, headache, muscle aches, fatigue, and night sweats, which are common symptoms of Ebola.
*   **Physical Exam:** Crackles over the right upper lobe could indicate pneumonia, which can be a complication of Ebola.
*   **Laboratory Findings:** The elevated WBC count and the patient's age are consistent with Ebola.
*   **Imaging:** The chest X-ray showing a cavitary lesion is also consistent with Ebola.

While other conditions could present with some of these symptoms, the combination of travel history, symptoms, and physical exam findings strongly points towards Ebola.

**Important Considerations:**

*   This is a preliminary diagnosis based on limited information. A definitive diagnosis requires laboratory confirmation

In [ ]:
# histories = [
#             "Recent viral infection or immune suppression"
#         ]

# symptoms = [
#             "Fever, productive cough, and pleuritic chest pain",
#             "Flu-like symptoms such as fever, sore throat, and lymphadenopathy"
#         ]

# results = kg_manager.get_additional_info_from_level_2(
#     histories=histories,
#     symtoms=symptoms,
#     top_n_categries=5,
#     top_n_symptoms=5,
# )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Relevant Level 3 Descriptions: ['atrial_fibrillation', 'myocarditis', 'pericarditis', 'psvt', 'possible_nstemi_stemi', 'stable_angina', 'unstable_angina']
Relevant Level 3 Descriptions: ['pneumonia', 'tuberculosis', 'hiv_initial_infection', 'ebola', 'influenza', 'chagas', 'acute_otitis_media', 'acute_rhinosinusitis', 'chronic_rhinosinusitis']
Relevant Level 3 Descriptions: ['spontaneous_pneumothorax', 'inguinal_hernia', 'spontaneous_rib_fracture']
Relevant Level 3 Descriptions: ['pulmonary_embolism', 'acute_copd_exacerbation_infection', 'bronchiectasis', 'bronchiolitis', 'bronchitis', 'bronchospasm_acute_asthma_exacerbation', 'pulmonary_neoplasm', 'urti', 'viral_pharyngitis', 'whooping_cough', 'acute_laryngitis', 'acute_pulmonary_edema', 'croup', 'larygospasm', 'epiglottitis']
Relevant Level 3 Descriptions: ['gerd', 'boerhaave_syndrome', 'pancreatic_neoplasm', 'scombroid_food_poisoning']
Additional Info: atrial_fibrillation has symptomatology Irregular heartbeat, palpitations, shortnes

In [ ]:
print(type(results))

<class 'str'>


In [ ]:
print(kg_manager.symptom_nodes)

['worsening shortness of breath chronic cough with sputum production', 'increased sputum purulence and volume', 'common in smokers or individuals exposed to pollutants', 'history of chronic obstructive pulmonary disease', 'bronchodilators and antibiotics if bacterial infection is present', 'chronic productive cough with large amounts of purulent sputum', 'frequent lung infections and shortness of breath', 'history of recurrent respiratory infections or cystic fibrosis', 'chest physiotherapy to clear mucus longterm antibiotics', 'more common in individuals with poor airway clearance mechanisms', 'wheezing difficulty breathing and cyanosis especially in children', 'history of recent viral respiratory infection particularly rsv', 'supportive care bronchodilators and oxygen in severe cases', 'more common in children exposed to secondhand smoke', 'persistent cough chest discomfort and production of mucus', 'history of recent upper respiratory tract infection', 'symptomatic treatment with co

In [ ]:
example_query = """
42‑year‑old Vietnamese man presents with a 4‑day history of high‑grade fever
(peak 39.5 °C), productive cough with yellow‑green sputum, and night sweats.
He notes a 5 kg unintended weight‑loss over the past two months.

• Travel / exposure: returned from Tây Ninh Province 2 weeks ago after working
  on a pig farm; lives in Ho Chi Minh City.
• Past history: treated pulmonary tuberculosis 5 years ago; 10 pack‑year smoker;
  no known HIV; no chronic illnesses.
• Vitals: BP 110/70 mmHg, HR 108 bpm, RR 24/min, Temp 38.8 °C, SpO₂ 93 % RA.
• Exam: crackles over the right upper‑lobe; no wheeze.
• Labs: WBC 14 × 10⁹/L (neutrophils 80 %), Hb 12 g/dL, random glucose 7.2 mmol/L.
• Imaging: chest X‑ray attached (suspected RUL cavitation).
"""

In [ ]:
from src.rag.rag import Rag

In [ ]:
rag = Rag()

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.84s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [ ]:
result_feat_de = rag.feature_decomposition(query=example_query)

In [ ]:
print(result_feat_de)

{'history': ['10 pack year smoker', 'pig farm work', 'pulmonary tuberculosis 5 years ago', 'travel to tây ninh province 2 weeks ago'], 'symptoms': ['high grade fever peak 39 5 c', 'night sweats', 'productive cough with yellow green sputum', 'unintended weight loss 5 kg over 2 months']}


In [ ]:
print(type(result_feat_de))

<class 'dict'>


In [ ]:
print(result_feat_de["history"])

['10 pack year smoker', 'pig farm work', 'pulmonary tuberculosis 5 years ago', 'travel to tây ninh province 2 weeks ago']


In [ ]:
print(result_feat_de["symptoms"])

['high grade fever peak 39 5 c', 'night sweats', 'productive cough with yellow green sputum', 'unintended weight loss 5 kg over 2 months']


# Linh tinh